<a href="https://colab.research.google.com/github/JoaoEmanuel14/ufs-ia-trabalho-a2/blob/main/IA_Trabalho_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explorando Modelos Multimodais: Como Texto, Áudio e Imagem se Combinam em IA

Repositório no GitHub: https://github.com/JoaoEmanuel14/ufs-ia-trabalho-a2

Dúvidas? Contate-nos em: joao.apostolo@dcomp.ufs.br

In [1]:
# Configurações de sistema utilizadas para fazer os experimentos
!nvidia-smi

Wed Apr  9 01:45:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1. Instalação de bibliotecas, imports e chaves das APIs

In [2]:
# Instalar bibliotecas necessárias para texto
!pip install openai google-generativeai -q
!pip install sentence-transformers

import openai
import google.generativeai as genai
from google.colab import userdata  # Para acessar as chaves armazenadas no Secrets
from sentence_transformers import SentenceTransformer, util

# Instalar bibliotecas necessárias para imagem
import torch
from PIL import Image
from IPython.display import display

# Instalar bibliotecas necessárias para áudio
!pip install openai vosk gtts -q
!pip install pydub

import wave
import time
import json
import io
import os
import re
import base64
import pydub
import urllib.request
import zipfile
from gtts import gTTS
from glob import glob
from vosk import Model, KaldiRecognizer
from IPython.display import Audio
from pydub import AudioSegment

# Pegar conteúdo do Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configurar chaves de API usando Secrets do Colab
try:
  openai_api_key = userdata.get('OPENAI_API_KEY')  # Chave da OpenAI
  google_api_key = userdata.get('GOOGLE_API_KEY')  # Chave do Google
  genai.configure(api_key=google_api_key)
except Exception as e:
  print("Erro ao carregar as chaves de API. Verifique se as chaves estão configuradas corretamente no Secrets.")
  raise e

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

## 2. Configuração dos clientes OpenAI e Google

### 2.1. Funções que processam áudio e imagem, gerando texto

In [3]:
# Configurar o cliente da OpenAI
client = openai.OpenAI(api_key=openai_api_key)


# Função para chamada ao GPT-4o com imagem e texto
def gerar_texto_openai(imagem_path, texto):
  """Esta função é responsável por gerar um resumo textual a partir
  de uma imagem e de outro texto, com o ChatGPT

  Args:
    imagem_path: Imagem que será processada
    texto: Obtido a partir de um áudio

  Returns:
    Resumo textual que corresponde as entradas
  """
  try:
    with open(imagem_path, "rb") as image_file:
      # Utilizando a biblioteca base64 para fazer a leitura da imagem
      imagem_base = base64.b64encode(image_file.read()).decode("utf-8")
      # É necessário transformar a imagem em URL para que o ChatGPT possa lê-la
      imagem_url = f"data:image/jpeg;base64,{imagem_base}"

      # Completion para o prompt
      response = client.chat.completions.create(
          model="gpt-4o",
          # Montando a mensagem
          messages=[
              {"role": "user", "content": [
                {"type": "text", "text": texto},
                {"type": "image_url", "image_url": {"url": imagem_url}}
              ]}
          ]
      )
      return response.choices[0].message.content

  except Exception as e:
    return f"Erro ao gerar texto com OpenAI: {str(e)}"


# Função para chamada ao Gemini com imagem e texto
def gerar_texto_google(imagem_path, texto):
  """Esta função é responsável por gerar um resumo textual a partir
  de uma imagem e de outro texto, com o Gemini

  Args:
    imagem_path: Imagem que será processada
    texto: Obtido a partir de um áudio

  Returns:
    Resumo textual que corresponde as entradas
  """
  try:
    model = genai.GenerativeModel("gemini-1.5-pro")
    imagem = Image.open(imagem_path)
    # Completion para o prompt
    response = model.generate_content([texto, imagem])
    return response.text

  except Exception as e:
    return f"Erro ao gerar texto com Google Gemini: {str(e)}"

### 2.2. Text-To-Speech (TTS): funções de conversão $texto \implies áudio$

In [4]:
# Função para conversão de texto em fala (TTS) com gTTS
def tts_gtts(texto):
  """Esta função é responsável por realizar a conversão de texto em fala (TTS)
  com gTTS

  Args:
    texto: Uma string dada pelo usuário

  Returns:
    Áudio correspondente ao texto inserido como entrada
  """
  try:
    # Criar o objeto gTTS
    tts = gTTS(texto, lang="pt")  # Define o idioma como pt-BR
    # Salvar o áudio em um arquivo MP3
    tts.save("gtts_audio.mp3")
    print("Áudio gerado e salvo como 'gtts_audio.mp3'.")
    return "gtts_audio.mp3"

  except Exception as e:
    print(f"Erro ao gerar áudio com gTTS: {str(e)}")
    return None


# Configurar o cliente da OpenAI
client = openai.OpenAI(api_key=openai_api_key)


# Função para conversão de texto em fala (TTS) com OpenAI
def tts_openai(texto):
  """Esta função é responsável por realizar a conversão de texto em fala (TTS)
  com o TTS da OpenAI

  Args:
    texto: Uma string dada pelo usuário

  Returns:
    Áudio correspondente ao texto inserido como entrada
  """
  try:
    # Completion para o prompt
    response = client.audio.speech.create(
        model="tts-1",
        input=texto,
        voice="alloy"
    )
    return response.content  # Retorna o conteúdo de áudio diretamente

  except Exception as e:
    print(f"Erro ao gerar áudio com OpenAI TTS: {str(e)}")
    return None

### 2.3. Função auxiliar que converte áudio do formato `mp3` para `wav`

In [5]:
# Função que converte mp3 para wav
def converter_mp3_para_wav(mp3_path, wav_path="audio.wav"):
  """A função é responsável por converter um aúdio em ".mp3" para ".wav", para
  que os modelos possam processar o áudio

  Args:
    mp3_path: Áudio em formato ".mp3"
    wav_path: Nome do arquivo em que o áudio convertido deve ser salvo

  Returns:
    wav_path: Áudio salvo em formato ".wav"
  """
  try:
    audio = AudioSegment.from_mp3(mp3_path)
    # Colocando o áudio no padrao PCM 16kHz mono, 16-bit
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    # Exportando o áudio no formato correto
    audio.export(wav_path, format="wav")
    print(f"Conversão concluída: '{wav_path}'")
    return wav_path

  except Exception as e:
    print(f"Erro na conversão de MP3 para WAV: {e}")
    return None

### 2.4. Função para transcrição de áudio usando Vosk

In [6]:
# Transcrever áudio usando Vosk
def transcrever_audio(caminho_audio):
  """Função responsável por transcrever um áudio

  Args:
    caminho_audio: O caminho do áudio que deve ser transcrito

  Returns:
    texto: Texto transcrito a partir do áudio de entrada
  """
  try:
    # Verifica se o modelo de reconhecimento de voz já foi baixado
    if not os.path.exists("vosk-model-small-pt-0.3"):
      print("Baixando o modelo...")
      # Faz o download do modelo de voz em português
      urllib.request.urlretrieve(
          "https://alphacephei.com/vosk/models/vosk-model-small-pt-0.3.zip",
          "vosk-model-small-pt-0.3.zip"
      )
      # Extrai os arquivos do modelo compactado
      with zipfile.ZipFile("vosk-model-small-pt-0.3.zip", "r") as zip_ref:
        zip_ref.extractall(".")

    wf = wave.open(caminho_audio, "rb")

    # Verifica se o áudio está no formato correto (mono, 16-bit, 16kHz)
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() != 16000:
      print("Formato de áudio inválido para Vosk. Esperado: WAV 16kHz mono PCM.")
      return None

    # Carrega o modelo Vosk
    model = Model("vosk-model-small-pt-0.3")

    # Inicializa o reconhecedor com o modelo e a taxa de amostragem do áudio
    rec = KaldiRecognizer(model, wf.getframerate())
    results = []

    # Lê o áudio em blocos e processa com o reconhecedor
    while True:
      data = wf.readframes(4000) # Lê 4000 frames de cada vez
      if len(data) == 0:
        break
      # Se a transcrição parcial for feita
      if rec.AcceptWaveform(data):
        # Converte o resultado para dicionário
        result = json.loads(rec.Result())
        # Adiciona o texto transcrito à lista
        results.append(result.get("text", ""))

    # Adiciona a transcrição final (último trecho processado)
    final = json.loads(rec.FinalResult())
    results.append(final.get("text", ""))

    # Junta todos os trechos de texto em uma única string
    texto = " ".join(results).strip()

    # Retorna o texto final, caso este nao esteja vazio
    if texto:
      return texto
    # Se estiver vazio, é retornado None
    else:
      return None

  except Exception as e:
    print("Erro na transcrição de áudio:", e)
    return None

### 2.5. Manipulação do arquivo que será usado para a construção da narrativa


In [7]:
# Função responsável por gerar os a tupla de pares (imagem, áudio)
def gerar_pares_imagem_audio(pasta_imagens, pasta_audios_mp3):
  """Função responsável por gerar pares (imagem, audio)

  Args:
  pasta_imagens: Pasta que contém as imagens
  pasta_audios_mp3: Pasta que contém os áudios em formato ".mp3"
  """
  pares_imagem_audio = []

  # Ordena os arquivos para garantir a correspondência sequencial
  imagens = sorted(glob(os.path.join(pasta_imagens, "*.jpg")))
  audios_mp3 = sorted(glob(os.path.join(pasta_audios_mp3, "*.mp3")))

  # O processo é interrompido caso o tamanho da pasta de imagens e de áudios for diferente
  if len(imagens) != len(audios_mp3):
    print("Quantidade de imagens e áudios não bate!")
    return []

  # Gera os pares após converter o áudio para o formato correto
  for idx, imagem in enumerate(imagens):
    audio_mp3 = audios_mp3[idx]

    wav_path = f"audio_convertido_{idx + 1}.wav"
    # Converte os áudios de .mp3 para .wav
    audio_wav_convertido = converter_mp3_para_wav(audio_mp3, wav_path)

    if audio_wav_convertido:
      pares_imagem_audio.append((imagem, audio_wav_convertido))
    else:
      print(f"[{idx + 1}] Erro ao converter áudio {audio_mp3}.")

  return pares_imagem_audio

### 2.6. Acurácia com similaridade semântica

In [8]:
# Inicializar modelo de embeddings
modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # Uso de um modelo multilíngue

def calcular_similaridade_semantica(texto_esperado, texto_gerado):
  """Função responsável por calcular a similaridade semântica
  entre dois textos utilizando embeddings e cosseno de similaridade,
  utilizando o modelo de linguagem SentenceTransformer

  Args:
    texto_esperado (str): Texto de referência (esperado).
    texto_gerado (str): Texto gerado que será comparado com o texto de referência.

  Returns:
    float: Valor da similaridade semântica entre os textos,
    arredondado para 3 casas decimais.
    O valor varia entre -1 (completamente diferente) e 1 (idêntico).
  """
  embedding1 = modelo.encode(texto_esperado, convert_to_tensor=True)
  embedding2 = modelo.encode(texto_gerado, convert_to_tensor=True)

  similaridade = util.pytorch_cos_sim(embedding1, embedding2).item()
  return round(similaridade, 3)


def avaliar_acuracia_multimodal(descricao_esperada, descricao_gerada):
  """Função responsável por avaliar a acurácia de uma descrição gerada com base
  em uma descrição esperada usando similaridade semântica.

  Args:
    descricao_esperada (str): A descrição correta ou esperada (referência).
    descricao_gerada (str): A descrição produzida pelo modelo (gerada).

  Returns:
    float: Similaridade semântica entre a descrição esperada e a gerada.
  """
  similaridade = calcular_similaridade_semantica(descricao_esperada, descricao_gerada)
  print(f"Similaridade semântica: {similaridade}")
  return similaridade

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 3. Entrada Multimodal

Fornecer uma imagem e um áudio descrevendo a cena, e pedir ao modelo para gerar um resumo textual.

In [9]:
# Função para limitar a entrada de texto transcrita
def limitar_texto(texto, limite=16000):
  """Função responsável por limitar o número de caracteres de uma string
  a um valor máximo definido.

  Se o texto ultrapassar o limite especificado, ele será truncado e um sufixo
  "..." será adicionado ao final. Caso contrário, o texto original é retornado.

  Args:
    texto (str): Texto a ser limitado.
    limite (int, opcional): Número máximo de caracteres permitidos.
                            O padrão é 16.000 caracteres.

  Returns:
    str: Texto original, ou texto truncado seguido de reticências ("...")
          se exceder o limite.
  """
  if len(texto) > limite:
    return texto[:limite] + "..."

  return texto


# Entrada multimodal
def entrada_multimodal(caminho_imagem, caminho_audio_mp3):
  """ Função responsável por processar uma entrada multimodal composta por uma
  imagem e um áudio (em MP3) para gerar e avaliar resumos textuais
  com dois modelos de linguagem: ChatGPT e Gemini.

  A função realiza os seguintes passos:
  1. Converte o áudio de MP3 para WAV.
  2. Transcreve o áudio em texto.
  3. Cria um prompt com base na transcrição e na imagem fornecida.
  4. Envia o prompt para os modelos ChatGPT e Gemini, obtendo as respostas.
  5. Calcula o tempo de resposta e a similaridade semântica (acurácia) entre
  a transcrição e as respostas geradas.
  6. Imprime os resultados obtidos.

  Args:
    caminho_imagem (str): Caminho para a imagem a ser utilizada como parte da
    entrada multimodal.
    caminho_audio_mp3 (str): Caminho para o arquivo de áudio em formato MP3.

  Returns:
    None: A função imprime os resultados diretamente no console.
          Em caso de erro na conversão ou transcrição, a execução é interrompida
          com uma mensagem de erro.
  """
  # Conversão do áudio para o formato correto
  caminho_wav = converter_mp3_para_wav(caminho_audio_mp3)
  if not caminho_wav:
    print("Erro ao converter áudio")
    return

  # Gera o texto transcrito a partir do áudio
  texto_transcrito = limitar_texto(transcrever_audio(caminho_wav))
  if not texto_transcrito:
    print("Erro: Transcrição falhou!")
    return

  prompt = f"Aqui está uma descrição de áudio: {texto_transcrito}. Com base nesta descrição e na imagem fornecida, gere um resumo textual."

  inicio_chatgpt = time.time()
  resposta_chatgpt = gerar_texto_openai(caminho_imagem, prompt)
  fim_chatgpt = time.time()
  tempo_chatgpt = fim_chatgpt - inicio_chatgpt

  inicio_gemini = time.time()
  resposta_gemini = gerar_texto_google(caminho_imagem, prompt)
  fim_gemini = time.time()
  tempo_gemini = fim_gemini - inicio_gemini

  # Acurácia multimodal
  acuracia_chatgpt = calcular_similaridade_semantica(texto_transcrito, resposta_chatgpt)
  acuracia_gemini = calcular_similaridade_semantica(texto_transcrito, resposta_gemini)

  # Resultados
  print("\n--- Resultados ---")
  print("Texto transcrito:", texto_transcrito)
  print("\nChatGPT:")
  print("Resposta:", resposta_chatgpt)
  print(f"Tempo de resposta: {tempo_chatgpt:.2f} segundos")
  print(f"Acurácia (similaridade): {acuracia_chatgpt}")

  print("\nGemini:")
  print("Resposta:", resposta_gemini)
  print(f"Tempo de resposta: {tempo_gemini:.2f} segundos")
  print(f"Acurácia (similaridade): {acuracia_gemini}")

In [10]:
sentenca = "Uma praia bonita. Possui coqueiros e água cristalina."
audio_gtts = tts_gtts(sentenca)
praia = '/content/drive/MyDrive/Inteligência Artificial/praia.jpg'
entrada_multimodal(praia, audio_gtts)

Áudio gerado e salvo como 'gtts_audio.mp3'.
Conversão concluída: 'audio.wav'
Baixando o modelo...

--- Resultados ---
Texto transcrito: uma praia bonita possui coqueiros e água cristalina

ChatGPT:
Resposta: A imagem mostra uma praia paradisíaca com areia branca, cercada por exuberantes coqueiros, criando um ambiente tropical relaxante. O mar é de um azul cristalino, sugerindo águas limpas e convidativas. No horizonte, há um trecho de vegetação verdejante cobrindo as colinas, compondo um cenário natural deslumbrante. O céu está parcialmente nublado, mas iluminado, contribuindo para o clima tranquilo e sereno do local.
Tempo de resposta: 4.21 segundos
Acurácia (similaridade): 0.887

Gemini:
Resposta: Uma praia de areia branca imaculada encontra-se na sombra de coqueiros inclinados, suas folhas verdes vibrantes contrastando com o céu azul e as nuvens brancas fofas. A água cristalina cintila sob o sol, a cor mudando de um azul-turquesa brilhante para um azul profundo à distância. Ao longe

## 4. Transferência de modalidade
Fornecer apenas a imagem e pedir ao modelo para gerar um áudio descritivo.

In [11]:
# Transferência de modalidade
def transferencia_modalidade(imagem_path, descricao_referencia):
  """Função responsável por realizar a tarefa de transferência de modalidade,
  onde uma imagem é descrita em texto e posteriormente convertida em áudio por
  modelos multimodais (ChatGPT e Gemini).

  A função executa os seguintes passos:
  1. Gera uma descrição textual da imagem fornecida, utilizando os modelos
  ChatGPT e Gemini.
  2. Compara semanticamente as descrições geradas com uma descrição de
  referência, calculando a acurácia.
  3. Converte as descrições geradas em arquivos de áudio.
  4. Exibe os tempos de resposta e as acurácias das respostas dos modelos.
  5. Reproduz os áudios gerados no final do processo.

  Args:
    imagem_path (str): Caminho para a imagem que será usada como entrada visual.
    descricao_referencia (str): Texto que representa a descrição esperada da
    imagem, usado como base para calcular a similaridade
    com as descrições geradas.

  Returns:
    None: A função imprime os resultados no console e exibe os áudios gerados.
          Em caso de erro na geração de descrição ou áudio,
          uma mensagem apropriada será exibida.
  """
  prompt = "Descreva esta imagem minuciosamente para que esta possa ser convertida em formato de áudio"

  print("Gerando descrição do ChatGPT...")
  inicio_chatgpt = time.time()
  descricao_chatgpt = gerar_texto_openai(imagem_path, prompt)
  fim_chatgpt = time.time()
  tempo_chatgpt = fim_chatgpt - inicio_chatgpt

  # Geração do áudio do ChatGPT a partir da descrição
  if descricao_chatgpt and not descricao_chatgpt.startswith("Erro"):
    print(f"Descrição gerada para áudio pelo ChatGPT: \n{descricao_chatgpt}")
    print(f"Tempo de resposta do ChatGPT: {tempo_chatgpt:.2f} segundos")

    # Acurácia multimodal
    acuracia_chatgpt = calcular_similaridade_semantica(descricao_referencia, descricao_chatgpt)
    print(f"Acurácia (ChatGPT): {acuracia_chatgpt}")

    # Geração e reprodução do áudio
    audio_chatgpt = tts_openai(descricao_chatgpt)
    if audio_chatgpt:
      with open("descricao_chatgpt.mp3", "wb") as f:
        # Salvando o áudio
        f.write(audio_chatgpt)
      print("Áudio salvo como 'descricao_chatgpt.mp3'")
      display(Audio("descricao_chatgpt.mp3", autoplay=True))
  else:
    print(f"Erro ao gerar descrição: {descricao_chatgpt}")

  print("\nGerando descrição do Gemini...")
  inicio_gemini = time.time()
  descricao_gemini = gerar_texto_google(imagem_path, prompt)
  fim_gemini = time.time()
  tempo_gemini = fim_gemini - inicio_gemini

  # Geração do áudio do Gemini a partir da descrição
  if descricao_gemini and not descricao_gemini.startswith("Erro"):
    print(f"Descrição gerada para áudio pelo Gemini: \n{descricao_gemini}")
    print(f"Tempo de resposta do Gemini: {tempo_gemini:.2f} segundos")

    # Acurácia multimodal
    acuracia_gemini = calcular_similaridade_semantica(descricao_referencia, descricao_gemini)
    print(f"Acurácia (Gemini): {acuracia_gemini}")

    # Geração e reprodução do áudio
    audio_gemini = tts_openai(descricao_gemini)
    if audio_gemini:
      with open("descricao_gemini.mp3", "wb") as f:
        # Salvando o áudio
        f.write(audio_gemini)
      print("Áudio salvo como 'descricao_gemini.mp3'")
      display(Audio("descricao_gemini.mp3", autoplay=True))
  else:
    print(f"Erro ao gerar descrição: {descricao_gemini}")

  # Resumo final
  print("\n--- Resumo dos tempos e acurácias ---")
  print(f"- ChatGPT: {tempo_chatgpt:.2f}s | Acurácia: {acuracia_chatgpt}")
  print(f"- Gemini:  {tempo_gemini:.2f}s | Acurácia: {acuracia_gemini}")

In [12]:
praia = '/content/drive/MyDrive/Inteligência Artificial/praia.jpg'
transferencia_modalidade(praia, "Uma praia bonita. Possui coqueiros e água cristalina.")

Gerando descrição do ChatGPT...
Descrição gerada para áudio pelo ChatGPT: 
A imagem retrata uma praia paradisíaca sob um céu azul com algumas nuvens brancas. À esquerda, há um grupo de palmeiras cujos longos troncos se inclinam em direção à areia clara da praia. As folhas verdes das palmeiras criam sombras que dançam sobre a areia. O mar à direita é de um azul cristalino, com ondas suaves que beijam a costa. À distância, é possível ver uma colina coberta por uma densa vegetação verde, criando um contraste com o azul intenso do oceano. O cenário transmite uma sensação de tranquilidade e beleza natural.
Tempo de resposta do ChatGPT: 4.05 segundos
Acurácia (ChatGPT): 0.773
Áudio salvo como 'descricao_chatgpt.mp3'



Gerando descrição do Gemini...
Descrição gerada para áudio pelo Gemini: 
A imagem apresenta uma idílica cena de praia tropical. Em primeiro plano, o foco está em uma areia branca e imaculada, levemente texturizada com pegadas e ondulações na superfície. Vários coqueiros imponentes dominam o lado esquerdo do primeiro plano, seus troncos grossos e inclinados projetados sobre a areia, criando áreas salpicadas de sombra. As folhas de palmeira verde-escuras e vibrantes são exuberantes e cheias, contrapondo-se ao céu azul brilhante.

O meio plano é uma faixa estreita de água turquesa vívida que gradualmente se torna mais escura à medida que se estende em direção ao horizonte. Pequenas ondas com cristas brancas quebradas rolam suavemente em direção à praia, indicando um mar calmo.

Ao fundo, uma colina verdejante e exuberante eleva-se da água, coberta por uma densa vegetação tropical. Ela parece uma pequena montanha ou uma grande colina, adicionando um elemento de drama à paisagem tranquila.


--- Resumo dos tempos e acurácias ---
- ChatGPT: 4.05s | Acurácia: 0.773
- Gemini:  9.18s | Acurácia: 0.73


## 5. Avaliação do Contexto

Fornecer uma sequência de imagens com descrições em áudio e verificar se o modelo consegue identificar a progressão da narrativa.

In [13]:
# Função para extrair índice do arquivo
def extrair_indice(path):
  """Função responsável por extrair o índice numérico de um nome de arquivo.

  A função busca por um número inteiro no nome do arquivo, como em
  'imagem_1.jpg' ou 'audio_2.wav', e retorna esse número como inteiro.
  Caso não encontre um número, retorna -1.

  Args:
    path (str): Caminho ou nome do arquivo contendo o índice numérico.

  Returns:
    int: Índice extraído do nome do arquivo ou -1 se nenhum número
    for encontrado.
  """
  match = re.search(r'(\d+)', path)
  return int(match.group(1)) if match else -1


# Função para ordenar os pares de acordo com o índice
def ordenar_pares_por_indice(pares_imagem_audio):
  """Função responsável por ordenar uma lista de pares (imagem, áudio)
  com base no índice extraído dos nomes dos arquivos.

  Esta função garante que os pares de imagem e áudio sejam organizados
  na ordem correta, com base no número presente nos nomes dos arquivos
  de imagem.

  Args:
    pares_imagem_audio (list[tuple]): Lista de tuplas (imagem_path, audio_path).

  Returns:
    list[tuple]: Lista ordenada de tuplas com base no índice numérico extraído
    do nome da imagem.
  """
  return sorted(pares_imagem_audio, key=lambda par: extrair_indice(par[0]))


# Avaliação do contexto para OpenAI ChatGPT
def avaliacao_do_contexto_openai(pares_imagem_audio, avaliacao_referencia):
  """Função respon'savel por avaliar a capacidade do modelo ChatGPT (OpenAI) de
  identificar progressão narrativa a partir de uma sequência de pares
  imagem-texto (áudio).

  A função executa os seguintes passos:
  1. Ordena os pares (imagem, áudio) com base no índice.
  2. Transcreve os áudios em texto e carrega as imagens em bytes.
  3. Constrói uma sequência multimodal com imagens e textos,
  simulando um diálogo com o modelo.
  4. Envia o prompt ao modelo GPT-4o, solicitando uma análise
  sobre a existência de narrativa progressiva.
  5. Calcula a similaridade semântica entre a resposta do modelo e
  a descrição de referência fornecida.
  6. Imprime os resultados, incluindo tempo de resposta e grau de acurácia.

  Parâmetros:
    pares_imagem_audio (list[tuple]): Lista de tuplas (imagem_path, audio_path),
    cada uma representando uma cena multimodal.
    avaliacao_referencia (str): Texto de referência esperado como avaliação da
    narrativa para comparação com a resposta do modelo.

  Retorno:
    None: A função imprime os resultados no console, incluindo a avaliação
    narrativa feita pelo modelo,
    tempo de resposta e acurácia calculada em relação à referência.
    Em caso de erro durante o processo, uma mensagem apropriada é exibida.
  """
  pares_ordenados = ordenar_pares_por_indice(pares_imagem_audio)

  textos_transcritos = []
  imagens_bytes = []

  print("\nTranscrevendo áudios e carregando imagens...")
  for idx, (imagem, audio) in enumerate(pares_ordenados):
    texto = transcrever_audio(audio)
    if texto:
      textos_transcritos.append((imagem, texto))
      try:
          with open(imagem, "rb") as imagem_file:
              imagens_bytes.append(imagem_file.read())
          print(f"[{idx + 1}] Texto transcrito: {texto}")
      except Exception as e:
          print(f"[{idx + 1}] Erro ao ler a imagem: {e}")
          return
    else:
      print(f"[{idx + 1}] Falha ao transcrever o áudio: {audio}")
      return

  print("\nAnalisando progressão narrativa com ChatGPT")
  inicio_chatgpt = time.time()

  # Mensagem inicial do prompt
  mensagens = [{
      "role": "system",
      "content": "Você é um especialista em análise narrativa. Avalie se há progressão lógica entre as imagens e as descrições fornecidas."
  }]

  # Montar a sequência multimodal para enviar ao modelo
  for idx, (imagem_bytes, (imagem_path, texto)) in enumerate(zip(imagens_bytes, textos_transcritos)):
      mensagens.append({
          "role": "user",
          "content": [
              {"type": "text", "text": f"Imagem {idx + 1}: {texto}"},
              {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64.b64encode(imagem_bytes).decode('utf-8')}"}}
          ]
      })

  mensagens.append({
      "role": "user",
      "content": "Com base nas imagens e nas descrições anteriores, existe uma narrativa progressiva?"
  })

  try:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=mensagens
    )
    resultado = response.choices[0].message.content
    fim_chatgpt = time.time()
    tempo_chatgpt = fim_chatgpt - inicio_chatgpt

    print("Avaliação do contexto (ChatGPT):")
    print(resultado)
    print(f"Tempo de resposta do ChatGPT: {tempo_chatgpt:.2f} segundos")

    # Acurácia multimodal
    acuracia_gpt = calcular_similaridade_semantica(avaliacao_referencia, resultado)
    print(f"Acurácia multimodal (ChatGPT): {acuracia_gpt}")

  except Exception as e:
    print(f"Erro ao analisar a narrativa: {e}")
    return None


# Avaliação do contexto para Google Gemini
def avaliacao_do_contexto_google(pares_imagem_audio, avaliacao_referencia):
  """Função responsável por avaliar a capacidade do modelo Google Gemini em
  identificar a progressão narrativa a partir de uma sequência de
  pares imagem-áudio.

  A função realiza as seguintes etapas:
  1. Ordena os pares (imagem, áudio) conforme o índice nos nomes dos arquivos.
  2. Transcreve os áudios em texto e carrega as imagens como bytes.
  3. Monta um prompt multimodal com pares de imagem e descrição textual.
  4. Envia o conteúdo ao modelo Gemini, solicitando uma análise da progressão
  narrativa.
  5. Compara a resposta do modelo com uma avaliação de referência utilizando
  similaridade semântica.
  6. Exibe a resposta do modelo, tempo de execução e grau de acurácia obtido.

  Args:
    pares_imagem_audio (list[tuple]): Lista de tuplas (imagem_path, audio_path)
    representando as cenas multimodais.
    avaliacao_referencia (str): Texto de avaliação esperado para comparação
    com a resposta do modelo.

  Returns:
    None: A função exibe os resultados no console, incluindo a avaliação gerada,
    tempo de resposta do Gemini e acurácia da análise. Em caso de erro,
    exibe uma mensagem informativa.
  """
  pares_ordenados = ordenar_pares_por_indice(pares_imagem_audio)

  textos_transcritos = []
  imagens_bytes = []

  print("\nTranscrevendo áudios e carregando imagens...")
  for idx, (imagem, audio) in enumerate(pares_ordenados):
    texto = transcrever_audio(audio)
    if texto:
      textos_transcritos.append((imagem, texto))
      try:
          with open(imagem, "rb") as imagem_file:
              imagens_bytes.append(imagem_file.read())
          print(f"[{idx + 1}] Texto transcrito: {texto}")
      except Exception as e:
          print(f"[{idx + 1}] Erro ao ler a imagem: {e}")
          return
    else:
      print(f"[{idx + 1}] Falha ao transcrever o áudio: {audio}")
      return

  print("\nAnalisando progressão narrativa com Gemini")
  inicio_gemini = time.time()

  try:
    # Carrega o modelo do Google Gemini
    model = genai.GenerativeModel("gemini-1.5-pro")
  except Exception as e:
    print(f"Erro ao inicializar o modelo Google Gemini: {str(e)}")
    return

  # Monta a entrada multimodal
  partes_entrada = [
      "Você é um especialista em análise narrativa. Avalie a sequência de imagens e descrições fornecidas.",
      "Determine se existe uma progressão narrativa lógica entre os eventos.\n"
  ]

  # Coloca os pares (imagem, texto) no prompt
  for idx, (imagem_bytes, (imagem_path, texto)) in enumerate(zip(imagens_bytes, textos_transcritos)):
    partes_entrada.append(f"Imagem {idx + 1}: {texto}")
    partes_entrada.append({"mime_type": "image/jpeg", "data": imagem_bytes})

  # Pergunta final
  partes_entrada.append("Com base nas imagens e nas descrições anteriores, existe uma narrativa progressiva?")

  try:
    # Completion para o modelo
    resultado = model.generate_content(partes_entrada)
    fim_gemini = time.time()
    tempo_gemini = fim_gemini - inicio_gemini

    print("Avaliação do contexto (Gemini):")
    print(resultado.text)
    print(f"Tempo de resposta do Gemini: {tempo_gemini:.2f} segundos")

    # Acurácia multimodal
    acuracia_gemini = calcular_similaridade_semantica(avaliacao_referencia, resultado.text)
    print(f"Acurácia multimodal (Gemini): {acuracia_gemini}\n")

  except Exception as e:
    print(f"Erro ao gerar análise com Gemini: {e}")
    return None

In [14]:
# Carregando as imagens e áudios do Google Drive
pasta_imagens = '/content/drive/MyDrive/Inteligência Artificial/Imagens_narrativa'
pasta_audios = '/content/drive/MyDrive/Inteligência Artificial/Audios_narrativa'
pares_imagens_audios = gerar_pares_imagem_audio(pasta_imagens, pasta_audios)

narrativa = "Sim, existe uma narrativa. Ela é a seguinte: Num dia em que o sol brilhava com força, um gato cinza chamado Tobias decidiu sair para explorar o bairro. Ele andava como se tivesse um compromisso sério — cauda erguida, passos firmes. No meio do caminho, encontrou algo incomum: um guarda-chuva azul, aberto, no meio da calçada. Tobias parou. Olhou para o guarda-chuva. Depois para o céu sem nuvens. “Humano distraído”, pensou ele. Mas ao tocar no guarda-chuva com a pata, ele se fechou de repente... e um portal se abriu! Tobias foi sugado para dentro com um miau! surpreso. Quando abriu os olhos, estava numa cidade onde todos os animais usavam roupas e andavam de patinete. Um coelho de óculos escuros o olhou e disse: — Seja bem-vindo a Umbrellópolis. Aqui, só entra quem encontra o Guarda-Chuva Azul. Tobias deu um miado resignado. Aparentemente, a aventura do dia estava só começando."

avaliacao_do_contexto_openai(pares_imagens_audios, narrativa)
avaliacao_do_contexto_google(pares_imagens_audios, narrativa)

Conversão concluída: 'audio_convertido_1.wav'
Conversão concluída: 'audio_convertido_2.wav'
Conversão concluída: 'audio_convertido_3.wav'
Conversão concluída: 'audio_convertido_4.wav'
Conversão concluída: 'audio_convertido_5.wav'

Transcrevendo áudios e carregando imagens...
[1] Texto transcrito: o gato tobias está explorando o bairro
[2] Texto transcrito: o gato tobias acham guarda chuva do
[3] Texto transcrito: o guarda chuva se fecha e abrir um portal
[4] Texto transcrito: o gato tobias sugado pelo portal
[5] Texto transcrito: o gato tobias chega em um bibelô polícia recebido pelo coelho estiloso

Analisando progressão narrativa com ChatGPT
Avaliação do contexto (ChatGPT):
Sim, há uma narrativa progressiva clara nas imagens e descrições:

1. **Imagem 1**: O gato Tobias está explorando o bairro, começando sua aventura.
2. **Imagem 2**: Tobias encontra um guarda-chuva misterioso.
3. **Imagem 3**: O guarda-chuva se fecha e abre um portal, surpreendendo Tobias.
4. **Imagem 4**: Tobias é